In [1]:
# Import packages
import os
import sys
import random
import time
from datetime import datetime


import pandas as pd
import requests

In [3]:
coingecko_token_names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        "REN"   : "ren",
        "RENFIL": "renfil",
        "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }

In [4]:
# Price Change Data
def priceDataChange():
    
    # Data Columns to be added in the datafram
    columns = []
    # Timestamp columns 
    columns.append('timestamp')
    
    # Iterating through coingecko token coingecko_token_names
    for key in list(coingecko_token_names.keys()):
        # adding price and 10 day change coloumn for each token by iterating over dict
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')

    priceChangeDF   = pd.DataFrame(columns = columns)
    
    # 2 Year Look back
    LOOK_BACK = 375 * 2
    #caching WBTC Price first as it will be used to extrapolate for tokens which does not historical data for a period
    coingecko_data      = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(coingecko_token_names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily").json()
    #Storing WBTC Prices
    WBTC      = coingecko_data['prices']
    time_list = []
    price_list= []
    # Selecting only 2 years of data. 
    # As prices are in acsending order based on time, latest 2 years are latest LOOK_BACK values
    for day in coingecko_data['prices'][-LOOK_BACK:]:
        # day = [timestamp, price at that timestamp]
        #As coingecko gives timestampin milliseconds, divinding it by 1000 to convert into Seconds to convert it back to date.
        time_list.append(day[0]/1000)
        price_list.append(day[1])
    
    # Appending WBTC prices and timestamp to dataframe.
    priceChangeDF['WBTC_price'] = price_list
    priceChangeDF['timestamp']  = time_list

    #Variable to keep track of period of extrapolated data for each token to use to replace percent change too.
    track = {}
    for asset in list(coingecko_token_names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            # Skipping WBTC as it's being cached
            continue
        
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(coingecko_token_names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        # Variable to store required WBTC Price
        token_price_holder    = []
        if request.status_code != 200:
            data = {}
            data['prices'] = WBTC[ -LOOK_BACK: ]
            for day in data['prices']:
                token_price_holder.append(day[1])
        else:   
            data = request.json()
            time_list   = []
            
            # Checking the length of available data for each token and calculating and computing extrapolation
            if len(data['prices']) < LOOK_BACK:
                # length of required WBTC Cached value
                req = LOOK_BACK - len(data['prices'])
                # selecting the token's missing dataframe from Cached WBTC prices
                addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
                data['prices'] = addition + data['prices']
                # Storing the count of values superimposed for each token
                track[asset] = req
            else:
                # if token have 2 years of historical prices, then just select that.
                data['prices'] = data['prices'][-LOOK_BACK:]

            for day in data['prices']:
                time_list.append(day[0]/1000)
                token_price_holder.append(day[1])
                
        priceChangeDF[asset+'_price'] = token_price_holder
        # priceChangeDF['timestamp']  = time_list
    # priceChangeDF['timestamp'] = time_list
    
    print("PRICES FETECHED")
    
    unknown_assets = ['PAX']
    for asset in list(coingecko_token_names.keys()):
#         try:
        #list variable for storing price change 
        change = []
        # Rolling difference of prices for each asset for 10 days
        diff_10 = pd.DataFrame(priceChangeDF[asset+'_price'].diff(periods=10) )

        # Appending Nan for initial 10 values to match the list size to dataframe rows.
        for i in range(10):
            change.append('Nan')

        # Calculating percent change 
        for i in range(10,priceChangeDF.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / priceChangeDF.iloc[i-10][asset+'_price']) * 100)

        priceChangeDF[asset+'_%change(10)'] = change


        # Rolling difference of prices for each asset for 1 days
        diff_1 = pd.DataFrame(priceChangeDF[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,priceChangeDF.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / priceChangeDF.iloc[i-1][asset+'_price']) * 100)
        priceChangeDF[asset+'_%change(1)'] = change  
            
#         except Exception as e:
#             unknown_assets.append(asset)
#             print("Error ", asset, e)
    
    # If data doesn't exists for any asset, using WBTC percent change.
    for asset in unknown_assets:
        priceChangeDF[asset+'_%change(1)']  = priceChangeDF['WBTC_%change(1)'] 
        priceChangeDF[asset+'_%change(10)'] = priceChangeDF['WBTC_%change(10)'] 
    priceChangeDF['PAX_%change(10)'] = priceChangeDF['WBTC_%change(10)'] 
    priceChangeDF['PAX_%change(1)'] = priceChangeDF['WBTC_%change(1)'] 


    # using WBTC price change % using tracked variable to calculate price change % for assets which doesn't
    # had prices for a period
    for key in track.keys():
        priceChangeDF[key+'_%change(10)'][:track[key]] = priceChangeDF['WBTC_%change(10)'][:track[key]]

    
    # adding data columns using timestamp 
    priceChangeDF['date'] = priceChangeDF['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    # Reversing the dataframe so dates comes in descending order
    priceChangeDF  = priceChangeDF.iloc[::-1]
    # Reseting Index to prevent any indexing errors.
    priceChangeDF  = priceChangeDF.reset_index()
    priceChangeDF = priceChangeDF.drop(['index'], axis=1)
    # removing last 10 values as they have Nan
    priceChangeDF = priceChangeDF[:-10]
    return priceChangeDF

In [5]:
priceDataChangeDF = priceDataChange()

PRICES FETECHED


/Users/rahulkanojia/miniconda3/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
priceDataChangeDF

,timestamp,DAI_price,DAI_%change(10),GUSD_price,GUSD_%change(10),SUSD_price,SUSD_%change(10),TUSD_price,TUSD_%change(10),USDC_price,...,ENJ_%change(1),KNC_%change(1),MANA_%change(1),REN_%change(1),RENFIL_%change(1),SNX_%change(1),ZRX_%change(1),PAX_%change(1),PAX_%change(10),date
0,1.644481e+09,1.000890,-0.042482,1.001935,0.13399,44155.902958,14.607693,1.001746,0.013193,1.001406,...,-0.630855,-2.273835,-1.407032,-0.630855,0.413998,-0.630855,-0.529459,-0.630855,14.607693,2022-02-10
1,1.644451e+09,1.001216,0.090062,1.000774,0.288379,44436.231158,16.950682,1.000293,0.017985,1.001332,...,0.595851,3.639296,-4.052331,0.595851,-0.598308,0.595851,2.764712,0.595851,16.950682,2022-02-10
2,1.644365e+09,1.002058,0.003176,1.000646,-0.063811,44173.025651,15.655022,0.999867,-0.270741,1.000992,...,0.681403,-4.587233,5.878043,0.681403,2.731187,0.681403,0.763,0.681403,15.655022,2022-02-09
3,1.644278e+09,1.000358,-0.105051,1.002240,0.090605,43874.066431,16.072597,1.001035,-0.154021,0.999134,...,3.219493,-3.928976,-0.517243,3.219493,7.732692,3.219493,3.198965,3.219493,16.072597,2022-02-08
4,1.644192e+09,1.000714,-0.329715,1.000324,-0.0944,42505.601731,17.730547,1.002536,-0.289006,1.001614,...,2.43426,1.477098,8.906537,2.43426,0.645969,2.43426,0.490621,2.43426,17.730547,2022-02-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,1.581034e+09,1.002880,0.431779,0.995732,-1.847074,9770.708824,9.478452,1.000198,0.116062,1.002281,...,1.562581,1.562581,10.513782,1.562581,1.562581,1.562581,7.712407,1.562581,9.478452,2020-02-07
736,1.580947e+09,0.995050,-0.805521,1.011576,0.502568,9620.382529,12.173001,1.000264,-0.067477,1.000150,...,4.711096,4.711096,1.262592,4.711096,4.711096,4.711096,0.967878,4.711096,12.173001,2020-02-06
737,1.580861e+09,1.002105,0.521444,0.990907,-0.737615,9187.548272,10.331085,1.000274,0.058638,1.002667,...,-1.127963,-1.127963,6.087609,-1.127963,-1.127963,-1.127963,8.591939,-1.127963,10.331085,2020-02-05
738,1.580774e+09,0.997300,0.02221,0.997098,-0.877996,9292.362709,9.958048,1.003442,0.219448,1.003335,...,-0.528189,-0.528189,4.058905,-0.528189,-0.528189,-0.528189,-1.817716,-0.528189,9.958048,2020-02-04
